In [13]:
# ==============================================================================
# PROJECT: Kaggle Playground Series - S6E2 (Heart Disease)
# MISSION: The Trinity Stack (Adding CatBoost) - GPU Conflict Fixed
# ==============================================================================

!pip install catboost -q

import pandas as pd
import numpy as np
from google.colab import drive
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# --- STEP 1: MOUNT & LOAD ---
drive.mount('/content/drive')
TRAIN_PATH = '/content/drive/MyDrive/Nihal Data/kaggle/S6E1 - heart/train.csv'
TEST_PATH = '/content/drive/MyDrive/Nihal Data/kaggle/S6E1 - heart/test.csv'

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

# --- STEP 2: PREPARATION ---
target_mapping = {'Absence': 0, 'Presence': 1}
y = train['Heart Disease'].map(target_mapping)
X = train.drop(['id', 'Heart Disease'], axis=1)
X_test = test.drop(['id'], axis=1)

# Categorical Encoding
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category').cat.codes
    X_test[col] = X_test[col].astype('category').cat.codes

# --- STEP 3: DEFINING THE TRINITY ---

# 1. XGBoost (The Optimized Veteran)
xgb_model = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.0524,
    max_depth=4,
    subsample=0.817,
    colsample_bytree=0.509,
    tree_method='hist',
    device='cuda',
    random_state=42
)

# 2. LightGBM (The Fast Specialist)
lgb_model = LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=63,
    device='gpu',
    random_state=42,
    verbosity=-1
)

# 3. CatBoost (The New Expert)
cat_model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    task_type="GPU", # Using GPU
    verbose=0,
    random_seed=42
)

base_models = [
    ('xgb', xgb_model),
    ('lgb', lgb_model),
    ('cat', cat_model)
]

# --- STEP 4: THE META-LEARNER ---
# FIX: Set n_jobs=1 to prevent GPU locking conflicts
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(),
    cv=5,
    stack_method='predict_proba',
    n_jobs=1 # <--- CHANGED FROM -1 TO 1
)

# --- STEP 5: TRAINING ---
print("Training the Trinity Stack... (Sequential execution to protect GPU)")
stack_model.fit(X, y)

# --- STEP 6: SUBMISSION ---
test_probs = stack_model.predict_proba(X_test)[:, 1]
submission = pd.DataFrame({'id': test['id'], 'Heart Disease': test_probs})
submission.to_csv('submission_v9_trinity_stack.csv', index=False)

print("Trinity Stack submission 'submission_v9_trinity_stack.csv' generated.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training the Trinity Stack... (Sequential execution to protect GPU)
Trinity Stack submission 'submission_v9_trinity_stack.csv' generated.
